# 概要
1. [remove_bg](https://github.com/takecx/remove_bg)で入力人物画像の背景を切り取った画像とmask画像を生成する
2. [PIFu](https://github.com/shunsukesaito/PIFu)で`.obj`ファイルを生成する
3. [obj2schematic](https://github.com/takecx/obj2schematic)で`.obj`ファイルを`.schematic`ファイルに変換する

In [ ]:
input_image = 'data/sample.png'

# remove_bg
kernel_size = 5
iteration = 3

# obj2schematic
w_max = 100
h_max = 100

In [ ]:
import sys,os
input_basename = os.path.splitext(os.path.basename(input_image))[0]

## remove_bg

In [ ]:
remove_bg_path = os.path.join(os.path.abspath(''), 'remove_bg/FBA_Matting')
if not remove_bg_path in sys.path:
    sys.path.append(remove_bg_path)

In [ ]:
!python remove_bg/BackgroundRemover.py $input_image --kernel_size $kernel_size --iteration $iteration

## PIFu

In [ ]:
# Training
GPU_ID=0
NAME='pifu_demo'

# Network configuration
BATCH_SIZE=1
MLP_DIM='257 1024 512 256 128 1'
MLP_DIM_COLOR='513 1024 512 256 128 3'

# Reconstruction resolution
# NOTE: one can change here to reconstruct mesh in a different resolution.
VOL_RES=256

CHECKPOINTS_NETG_PATH='./PIFu/checkpoints/net_G'
CHECKPOINTS_NETC_PATH='./PIFu/checkpoints/net_C'

TEST_FOLDER_PATH='./output'

# command
!CUDA_VISIBLE_DEVICES=$GPU_ID python ./PIFu/apps/eval.py \
    --name $NAME \
    --batch_size $BATCH_SIZE \
    --mlp_dim $MLP_DIM \
    --mlp_dim_color $MLP_DIM_COLOR \
    --num_stack 4 \
    --num_hourglass 2 \
    --resolution $VOL_RES \
    --hg_down 'ave_pool' \
    --norm 'group' \
    --norm_color 'group' \
    --test_folder_path $TEST_FOLDER_PATH \
    --load_netG_checkpoint_path $CHECKPOINTS_NETG_PATH \
    --load_netC_checkpoint_path $CHECKPOINTS_NETC_PATH

## obj2schematic

In [ ]:
pifu_obj = os.path.join('./results/pifu_demo/', 'result_' + input_basename + '_fg.obj')

In [ ]:
!python obj2schematic/Obj2SchemticConverter.py $pifu_obj  --h_max $h_max --w_max $w_max